In [1]:
import numpy as np
import pandas as pd

def as_colvec(x):   
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x,axis=1)

In [2]:
np.arange(4)

array([0, 1, 2, 3])

In [3]:
as_colvec (np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

In [4]:
def implied_returns(delta,sigma,w):    #delta= risk aversion coefficient, sigma= var covar matrix, w=portfolio weights
    ir = delta* sigma.dot(w).squeeze()
    ir.name = 'Implied Returns'
    return ir

In [5]:
def proportional_prior(sigma,tau,p):
    helit_omega = p.dot(tau*sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

In [6]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q, 
       omega = None,
       delta=7.5, tau=.20):

    if omega is None:
         omega = proportional_prior(sigma_prior, tau, p)
    
    N = w_prior.shape[0]
    
    K = q.shape[0]
    
    pi = implied_returns(delta, sigma_prior, w_prior)
    
    sigma_prior_scaled = tau * sigma_prior
    
    nu_bl = nu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+ omega).dot(q-p.dot(pi).values))
    ##nu_bl = pi + (p.T).dot(sigma_prior_scaled).dot(inv(sigma_prior_scaled.dot(p)).dot(p.T)+ omega).dot(q-p.dot(pi).values)
    #nu_bl = pi + (p.T)@(sigma_prior_scaled)@(inv(sigma_prior_scaled@(p))@(p.T)+ omega)@(q-p@(pi).values)

    
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+ omega)).dot(p).dot(sigma_prior_scaled)
    ##sigma_bl = sigma_prior + sigma_prior_scaled - (p.T).dot(sigma_prior_scaled).dot(inv(sigma_prior_scaled.dot(p).dot(p.T)+ omega)).dot(sigma_prior_scaled).dot(p)
    
    #sigma_bl = sigma_prior + sigma_prior_scaled - (p.T)@(sigma_prior_scaled)@(inv(sigma_prior_scaled@ p @(p.T)+ omega))@(sigma_prior_scaled)@p

    return (nu_bl, sigma_bl)


In [7]:
prices = pd.read_excel('prices.xlsx',header=0,index_col=0, parse_dates= True, usecols="A:H")
returns = prices.pct_change()
returns = returns.dropna()

In [8]:
#returns= pd.DataFrame(returns)

#tickers = returns[:0] 
tickers = ['Euro Gov', 'Greek Gov', 'EU Corporate', 'EU HY' , 'Eur Eq', 'US Eq', 'Cash']
s = returns.cov()*52
s = np.matrix(s)
s = pd.DataFrame(s,index=tickers, columns=tickers)

In [9]:

#input the weights of the benchmark
WEuroGov = .30
WGreekGov = .10
WEUCorporate = .20
WEUHY = .05
WEurEq = .10
WUSEq = .10
WCash = .15

weights = pd.Series([WEuroGov, WGreekGov, WEUCorporate, WEUHY, WEurEq, WUSEq, WCash],index= tickers)
#weights = np.matrix(weights)
#weights = pd.DataFrame(weights)

pi=implied_returns(delta=7.5,sigma=s, w=weights)
pi


Euro Gov        0.004326
Greek Gov       0.069389
EU Corporate    0.002424
EU HY           0.017458
Eur Eq          0.052837
US Eq           0.045285
Cash           -0.000062
Name: Implied Returns, dtype: float64

In [10]:
from scipy.optimize import minimize

def inverse(d):
    
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def portfolio_return (weights, mu):
    
    return weights.T @ mu

def portfolio_vol(weights, s):
    
    return (weights.T @ s@ weights)**0.5

def w_msr( mu, s):
    
    n = mu.shape[0]
    init_guess = np.repeat(1/n,n)
    bounds = [(0.,1.) for i in range(n)]
    weights_sum_to_1 = {
        
        'type': 'eq',
        'fun': lambda weights: np.sum(weights)-1
    }
    
    def neg_sharpe_ratio(weights, mu, s):
        
        r = portfolio_return(weights,mu)
        vol = portfolio_vol(weights, s)
        return -r/vol
    
    results = minimize(neg_sharpe_ratio, init_guess,
                      args=(mu, s,), method = "SLSQP",
                      options={'disp':False},
                      constraints= (weights_sum_to_1),
                      bounds = bounds
                      )
    return results.x
                     
    

In [11]:
#input the expectations
EEuroGov = .0
EGreekGov = .05
EEUCorporate = .04
EEUHY = .06
EEurEq = .04
EUSEq = .07
ECash = .0

mu_exp = pd.Series([EEuroGov, EGreekGov, EEUCorporate, EEUHY, EEurEq, EUSEq, ECash], index=tickers)
np.round(w_msr( mu_exp, s)*100,2)

array([ 0.  ,  0.  , 64.11,  0.  ,  0.  ,  1.17, 34.73])

In [12]:
q= pd.Series(mu_exp)

p= pd.DataFrame([
    {'Euro Gov':1, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':1, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':1, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':1, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':1, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':1, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':1}
    ])


In [13]:
bl_mu, bl_sigma = bl(w_prior= weights, sigma_prior=s, p=p, q=q)

bl_mu

Euro Gov        0.013251
Greek Gov       0.074091
EU Corporate    0.019528
EU HY           0.057362
Eur Eq          0.064274
US Eq           0.054780
Cash           -0.000085
dtype: float64

In [14]:
w_msr( bl_mu, bl_sigma)

array([5.66588902e-16, 3.57167304e-03, 6.87413766e-01, 2.52186209e-02,
       2.46613135e-16, 1.68798566e-02, 2.66916084e-01])